In [1]:
import collections
import json
import random
import re
from itertools import chain
from multiprocessing import Pool
from pathlib import Path
from typing import *

import git
import pandas as pd
import pydriller
from pymongo import MongoClient

In [2]:
import dotenv
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])
MVN_ECO_DIR = Path(ENV["MVN_ECO_DIR"])
DATA_DIR, DATA_DIR.exists(), MVN_ECO_DIR, MVN_ECO_DIR.exists()

(PosixPath('/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data'),
 True,
 PosixPath('/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/MavenEcoSysResearch-main'),
 True)

In [3]:
mvn_eco_dep_data = pd.read_csv(MVN_ECO_DIR/'csv'/'DEP.csv').to_dict(orient='records')
mvn_eco_patch_data = pd.read_csv(MVN_ECO_DIR/'csv'/'PATCH.csv').to_dict(orient='records')
mvn_eco_cve_data = pd.read_csv(MVN_ECO_DIR/'csv'/'CVE.csv').to_dict(orient='records')
mvn_eco_dep_data[0].keys(), mvn_eco_patch_data[0].keys(), mvn_eco_cve_data[0].keys()

(dict_keys(['Upstream G:A:V', 'Downstream G:A:V']),
 dict_keys(['CVE_ID', 'PATCH', 'G:A:V']),
 dict_keys(['CVE_ID', 'CVSS', 'CWE', 'VUL_FUNs']))

In [4]:
len(list(x['CVE_ID'] for x in mvn_eco_cve_data)), len(set(x['CVE_ID'] for x in mvn_eco_cve_data)), len(list(x['CVE_ID'] for x in mvn_eco_patch_data)), len(set(x['CVE_ID'] for x in mvn_eco_patch_data))

(832, 832, 832, 832)

In [5]:
mvn_eco_cve_dict = dict([(x['CVE_ID'], x) for x in mvn_eco_cve_data])
mvn_eco_patch_dict = dict([(x['CVE_ID'], x) for x in mvn_eco_patch_data])
len(set(mvn_eco_cve_dict.keys()).intersection(set(mvn_eco_patch_dict.keys())))

832

In [6]:
records = list()
for cve_id in mvn_eco_cve_dict.keys():
    vuln_gav = mvn_eco_patch_dict[cve_id]['G:A:V']
    vuln_funs: list[str] = mvn_eco_cve_dict[cve_id]['VUL_FUNs'].split(";")
    vuln_classes = set()
    vuln_methods = set()
    for vf in vuln_funs:
        vc = vf.split(':')[0].strip()
        vc = vc.split('$')[0]
        
        vm = vf.split(':')[1].split('(')[0].strip()
        
        if vm == '<init>':
            vm = vc.split('.')[-1].strip()
        
        vm = vc.split('.')[-1].strip() + '::' + vm
        
        vuln_classes.add(vc)
        vuln_methods.add(vm)
    
    r = dict()
    r['data_src'] = 'mvn_eco'
    r['cve_ref'] = cve_id
    r['vuln_gav'] = vuln_gav
    r['vuln_classes'] = list(vuln_classes)
    # r['vuln_methods'] = list(vuln_methods)
    r['patches'] = mvn_eco_patch_dict[cve_id]['PATCH'].split(';')
    
    records.append(r)

In [7]:
random.choice(records)

{'data_src': 'mvn_eco',
 'cve_ref': 'CVE-2017-15695',
 'vuln_gav': 'org.apache.geode:geode-core:1.2.1',
 'vuln_classes': ['org.apache.geode.management.internal.cli.functions.MembersForRegionFunction',
  'org.apache.geode.security.ResourcePermission',
  'org.apache.geode.management.internal.cli.functions.GetRegionDescriptionFunction',
  'org.apache.geode.management.internal.cli.functions.GetMemberConfigInformationFunction',
  'org.apache.geode.management.internal.cli.functions.CloseDurableCqFunction',
  'org.apache.geode.internal.cache.execute.util.FindRestEnabledServersFunction',
  'org.apache.geode.management.internal.cli.functions.GetSubscriptionQueueSizeFunction',
  'org.apache.geode.management.internal.configuration.functions.RecreateCacheFunction',
  'org.apache.geode.management.internal.cli.functions.MemberRegionFunction',
  'org.apache.geode.management.internal.cli.functions.CreateAsyncEventQueueFunction',
  'org.apache.geode.management.internal.cli.functions.FetchSharedConfigur

In [8]:
client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio
db.mvnEcoVuln.drop()
_ = db.mvnEcoVuln.insert_many(records)